In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3,Case3T
import numpy as np
import pandas as pd

In [3]:
DIR = BASE_DIR/'scripts_res'/'Case3THyper3'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Case3THyper3')

In [4]:
sol = 'ASBO'
case_name = 'Case3T'
runs_number = 5
n_iter = 200
group = 7
tt = pd.read_csv(DIR/'LHS_Case3T.csv')
hyper_p_list = tt.loc[group*5:4+group*5,:]
hyper_p_list

,0,1,2,3
35,3.0,2.0,0.76,0.95
36,3.0,2.0,0.78,0.92
37,4.0,2.0,0.78,0.95
38,5.0,2.0,0.80,0.95
39,5.0,2.0,0.80,0.95


In [5]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

for i in range(5*group,5+group*5,1):
    hyper_p = hyper_p_list.loc[i,:]
    m1,m2,p1,p2 = hyper_p[0],hyper_p[1],hyper_p[2],hyper_p[3]
    print(m1,m2,p1,p2)
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        try:
            optimizer.maximize(init_points=100, n_iter=n_iter,p_hyper=[[m1,m2,p1,p2,1e-5]],
                                acquisition_function=acquisition_function)
            print(optimizer.max)
            x_1_list.extend([optimizer.max['params']['x1']])
            x_2_list.extend([optimizer.max['params']['x2']])
            x_3_list.extend([optimizer.max['params']['x3']])
            x_4_list.extend([optimizer.max['params']['x4']])
            x_5_list.extend([optimizer.max['params']['x5']])  
            obj_list.extend([optimizer.max['target']])                                  
            con_1_list.extend([optimizer.max['constraint'][0]])
            con_2_list.extend([optimizer.max['constraint'][1]])
            con_3_list.extend([optimizer.max['constraint'][2]])
            con_4_list.extend([optimizer.max['constraint'][3]])        
            all_res = [item['target'] for item in optimizer.res]
            all_fea = [item['allowed'] for item in optimizer.res]
            modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
            all_res_list.append(modified_res)        
        except Exception as e:
            print(e)
            obj_list.append(None)
            x_1_list.append(None)
            x_2_list.append(None)
            x_3_list.append(None)
            x_4_list.append(None)
            x_5_list.append(None)                        
            con_1_list.append(None)
            con_2_list.append(None)
            con_3_list.append(None)
            con_4_list.append(None) 
            modified_res = [-np.inf for _ in range(100+n_iter)]
            all_res_list.append(modified_res)       

        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'x_3': x_3_list,
        'x_4': x_4_list,
        'x_5': x_5_list,
        'con_1': con_1_list,
        'con_2': con_2_list,
        'con_3': con_3_list,
        'con_4': con_4_list
        })
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+n_iter))])
        df.to_csv(DIR/f'{sol}_Hyper_Results_{case_name}_{runs_number}_{group}.csv', index=False)
        df2.to_csv(DIR/f'{sol}_Hyper_Results2_{case_name}_{runs_number}_{group}.csv', index=False)

3.0 2.0 0.76 0.95
{'target': -4.694180883281273, 'params': {'x1': 4.00027556847319, 'x2': 0.03188400772786091, 'x3': -3.9971794099398914, 'x4': 1.9875794785676912, 'x5': -5.977740269345635}, 'constraint': array([-0.42593369, -0.6476267 ,  1.0132202 ,  0.69647938])}
{'target': -3.8664575863412236, 'params': {'x1': 1.0816047757631964, 'x2': -0.906453167888236, 'x3': 0.28872989428299084, 'x4': -0.7811603802623385, 'x5': 0.9133480481762999}, 'constraint': array([0.8770419 , 2.34372448, 2.75786935, 0.22322508])}
{'target': -0.3102287503598906, 'params': {'x1': -0.972629370719326, 'x2': 1.0651408109620366, 'x3': 1.0736099979308384, 'x4': 1.1386222364445358, 'x5': -0.8752423199932526}, 'constraint': array([0.41446981, 2.54735977, 2.20697371, 0.60641578])}
{'target': -0.1370405733696427, 'params': {'x1': -0.9965255013752986, 'x2': 0.017449599164475083, 'x3': 1.9547426289256056, 'x4': 1.174260061769734, 'x5': 0.950051539970231}, 'constraint': array([-0.54957006,  0.37367845,  2.0392231 ,  1.317